In [1]:
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import cnn_builder as cbuild
import cnn_methods as cfunc
import config
import csv
import helper_fxns as hf
import importlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
%matplotlib inline

Using TensorFlow backend.


In [21]:
importlib.reload(config)
importlib.reload(cfunc)
importlib.reload(hf)
importlib.reload(cbuild)
C = config.Config()

In [36]:
model = cbuild.build_2d_cnn(C, 'adam')#Adam(lr=0.01))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_69 (InputLayer)            (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
input_70 (InputLayer)            (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
input_71 (InputLayer)            (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
conv2d_9 (Conv2D)                (None, 34, 34, 64)    640         input_69[0][0]                   
___________________________________________________________________________________________

model = cbuild.run_cnn(model, C)

In [37]:
nb_classes = len(C.classes_to_include)
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)
#intensity_df.loc[intensity_df["art_int"] == 0, "art_int"] = np.mean(intensity_df[intensity_df["art_int"] > 0]["art_int"])
#intensity_df.loc[intensity_df["ven_int"] == 0, "ven_int"] = np.mean(intensity_df[intensity_df["ven_int"] > 0]["ven_int"])
#intensity_df.loc[intensity_df["eq_int"] == 0, "eq_int"] = np.mean(intensity_df[intensity_df["eq_int"] > 0]["eq_int"])

orig_data_dict, num_samples = cfunc.collect_unaug_data(C, voi_df, intensity_df)
print(num_samples)

avg_X2 = {}
for cls in C.classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

{'hemangioma': 56, 'cholangio': 56, 'cyst': 75, 'colorectal': 74, 'hcc': 90, 'fnh': 67}


In [39]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_num = C.classes_to_include.index(cls)

    train_samples[cls] = round(num_samples[cls]*C.train_frac)

    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])

    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]

    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]

    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))

#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

X_test = cfunc.separate_phases_2d(X_test)
X_train_orig = cfunc.separate_phases_2d(X_train_orig)

hemangioma has 45 samples for training (2250 after augmentation) and 11 for testing
cholangio has 45 samples for training (2250 after augmentation) and 11 for testing
cyst has 60 samples for training (3000 after augmentation) and 15 for testing
colorectal has 59 samples for training (2950 after augmentation) and 15 for testing
hcc has 72 samples for training (3600 after augmentation) and 18 for testing
fnh has 54 samples for training (2700 after augmentation) and 13 for testing


train_generator = train_generator_func()
model_pretrain.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

In [40]:
#early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)
train_generator = cbuild.train_generator_func_2d(C, train_ids, intensity_df, voi_df, avg_X2, n=6, n_art=4)
model.fit_generator(train_generator, steps_per_epoch=40, epochs=30)#, callbacks=[early_stopping])

Epoch 1/30
40/40 [==============================] - 14s - loss: 1.4733 - acc: 0.3546    
Epoch 2/30
40/40 [==============================] - 13s - loss: 1.1141 - acc: 0.5338    
Epoch 3/30
40/40 [==============================] - 13s - loss: 0.9240 - acc: 0.6225    
Epoch 4/30
40/40 [==============================] - 13s - loss: 0.8386 - acc: 0.6563    
Epoch 5/30
40/40 [==============================] - 13s - loss: 0.7930 - acc: 0.6842    
Epoch 6/30
40/40 [==============================] - 13s - loss: 0.7531 - acc: 0.7050    
Epoch 7/30
40/40 [==============================] - 13s - loss: 0.7139 - acc: 0.7246    
Epoch 8/30
40/40 [==============================] - 13s - loss: 0.6881 - acc: 0.7225    
Epoch 9/30
40/40 [==============================] - 13s - loss: 0.6611 - acc: 0.7446    
Epoch 10/30
40/40 [==============================] - 13s - loss: 0.6569 - acc: 0.7388    
Epoch 11/30
40/40 [==============================] - 12s - loss: 0.5823 - acc: 0.7767    
Epoch 12/30
40/40 [

hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [41]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
    
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [35]:
cbuild.overnight_run(C)

6cls accuracy: 0.771084337349  - average: 0.771084337349
3cls accuracy: 0.819277108434  - average: 0.819277108434
6cls accuracy: 0.710843373494  - average: 0.740963855422
3cls accuracy: 0.771084337349  - average: 0.795180722892
6cls accuracy: 0.722891566265  - average: 0.734939759036
3cls accuracy: 0.759036144578  - average: 0.78313253012
6cls accuracy: 0.650602409639  - average: 0.713855421687
3cls accuracy: 0.722891566265  - average: 0.768072289157
6cls accuracy: 0.66265060241  - average: 0.703614457831
3cls accuracy: 0.78313253012  - average: 0.771084337349


KeyboardInterrupt: 

In [42]:
C.output_img_dir = "Z:\\OUTPUT\\12-08-2d"
C.classes_to_include

['hcc', 'cholangio', 'colorectal', 'cyst', 'hemangioma', 'fnh']

In [43]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_test, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C)

In [48]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_train_orig, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C, save_dir="Z:\\OUTPUT\\12-08-2d\\training")

In [54]:
cm

array([[ 7,  0,  0,  0,  0,  3],
       [ 0, 10,  1,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0],
       [ 0,  0,  0, 15,  0,  0],
       [ 0,  0,  0,  1, 10,  0],
       [ 0,  0,  0,  0,  0, 13]], dtype=int64)

In [45]:
y_true_simp, y_pred_simp, _ = cfunc.condense_cm(y_true, y_pred, C.classes_to_include)

In [46]:
accuracy_score(y_true_simp, y_pred_simp)

0.80722891566265065

In [44]:
accuracy_score(y_true, y_pred)

0.75903614457831325

4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%, 84%/86%

6 cls no artificial, separate phases: 75%/84%

6 cls 2d: 77%/84%, 74%/80%

fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

import operator
#print("Ground truth:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% probability)" % (max(x)*100) for x in Y_])